# Assignment 2
You will tackle with a sentiment classification task using LSTM model and attention mechanism in this assigment.

# Dependencies
Please make sure that you are using **GPU** to accelarate computation.

Colab FAQ: https://research.google.com/colaboratory/faq.html

## Import dependencies

In [2]:
import torch
import os
import collections
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from tqdm import tqdm
import math
import random
import numpy as np

In [3]:
# Set up your device 
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")

# The assertion is to make sure GPU is available
assert cuda == True

In [4]:
# Set up random seed to 1008. Do not change the random seed.
# Yes, these are all necessary when you run experiments!
seed = 1008
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


## Data
The script below will download the required sentiment analysis data.

Data folder will be visible in the Colab file-explorer pane, which is loacted at left side of the page.


In [11]:
!wget --no-check-certificate "https://docs.google.com/uc?export=download&id=1jqYJ9jhjukhXvEk4GnMAPYE-SvhSG24i" -O data.zip
!unzip data.zip

Archive:  data.zip
  inflating: sentiment/mapping.txt   
  inflating: sentiment/test_labels.txt  
  inflating: sentiment/test_text.txt  
  inflating: sentiment/train_labels.txt  
  inflating: sentiment/train_text.txt  
  inflating: sentiment/val_labels.txt  
  inflating: sentiment/val_text.txt  


## Corpus
Glove will be used as the word embedding tool in this assigment.

In [17]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

^C
Archive:  glove.6B.zip


  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in glove.6B.zip,
        and cannot find glove.6B.zip.zip, period.


# Preprocess
Preprocess data, then construct dataloader and vocabulary.

## Load Glove pretrained word embedding.

In [ ]:
# TODO

## Construct your own vocabulary without other corpus.
Hint: You should construct a vocabulary to map the word to index.

In [ ]:
# TODO

## Load data
Load data and construct dataloader.

In [ ]:
data_dir = 'sentiment'
# TODO

# Model
Bidirectional LSTM and attention mechanism will be used in this section.

## Model Zoo

In [ ]:
class BiRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, pretrained_embedding=None, **kwargs):
        super(BiRNN, self).__init__()
        if pretrained_embedding is None:
            self.embedding = nn.Embedding(vocab_size, embed_size)
        else:
            self.embedding= nn.Embedding.from_pretrained(torch.tensor(pretrained_embedding, dtype=torch.float), freeze=True)
        self.encoder = nn.LSTM(embed_size, num_hiddens, num_layers=num_layers, bidirectional=True, batch_first=True)
        self.decoder = nn.Sequential(nn.Linear(4 * num_hiddens, num_hiddens), nn.Linear(num_hiddens, 3))

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        self.encoder.flatten_parameters()
        outputs, _ = self.encoder(embeddings)
        encoding = torch.cat((outputs[:,0,:], outputs[:,-1,:]), dim=1)
        outs = self.decoder(encoding)
        return outs

In [ ]:
class BiRNN_attention(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, pretrained_embedding=None, **kwargs):
        super(BiRNN_attention, self).__init__()
        if pretrained_embedding is None:
            self.embedding = nn.Embedding(vocab_size, embed_size)
        else:
            self.embedding = nn.Embedding.from_pretrained(torch.tensor(pretrained_embedding, dtype=torch.float),
                                                          freeze=True)
        self.encoder = nn.LSTM(embed_size, num_hiddens, num_layers=num_layers, bidirectional=True, batch_first=True)
        self.weight_W = nn.Parameter(torch.Tensor(2 * num_hiddens, 2 * num_hiddens))
        self.weight_proj = nn.Parameter(torch.Tensor(2 * num_hiddens, 1))

        self.decoder = nn.Sequential(nn.Linear(2 * num_hiddens, num_hiddens), nn.Linear(num_hiddens, 3))
        nn.init.uniform_(self.weight_W, -0.1, 0.1)
        nn.init.uniform_(self.weight_proj, -0.1, 0.1)


    def forward(self, inputs):
        mask = 1 - torch.clamp(inputs, min=0, max=1)
        embeddings = self.embedding(inputs)
        states, hidden = self.encoder(embeddings.permute([0, 1, 2]))
        u = torch.tanh(torch.matmul(states, self.weight_W))
        att = torch.matmul(u, self.weight_proj)
        att = att + mask.unsqueeze(2) * -1e7
        att_score = F.softmax(att, dim=1)
        scored_x = states * att_score
        encoding = torch.sum(scored_x, dim=1)
        outputs = self.decoder(encoding)

        return outputs

## Training
You should train two models above with Glove pretrained word embedding and random initialized word embedding.

Evaluation on the validation set and print out accuracy after training one epoch is required.

You can tune some parameters and try different techniques, such as learning rate scheduler.

In [ ]:
# Train BiRNN with Glove pretrained word embedding
# TODO

In [ ]:
# Train BiRNN without pretrained word embedding
# TODO

In [ ]:
# Train BiRNN_attention with Glove pretrained embedding
# TODO

In [ ]:
# Train BiRNN_attention without pretrained word embedding
# TODO

# Report (optional)
You can briefly report what strategies you attempted in this assignment.